<a href="https://colab.research.google.com/github/boscoybarra/customer-transaction-prediction/blob/master/customer_transaction_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading Kaggle Dataset and dependencies


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

In [0]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

In [0]:
# Install kaggle pip
!pip install -U -q kaggle

!mkdir ~/.kaggle/

# We copy the json file into a apparently temp file .kaggle
!cp "/content/drive/My Drive/kaggle/kaggle.json" ~/.kaggle/




In [0]:
!chmod 600 /root/.kaggle/kaggle.json

##Download the data 

In [0]:
!kaggle competitions download santander-customer-transaction-prediction
!ls



#Starting point


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics

import matplotlib.pyplot as plt
from scipy import stats

In [0]:
train = pd.read_csv('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/train.csv')
test = pd.read_csv('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/test.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/sample_submission.csv')


In [0]:
train

In [0]:
# targets = train['targets']
# target_transactions = targets.sum(axis=None)
# print(target_transactions)

train.shape

In [0]:
train

In [0]:
pre_transactions = train[train.columns[1:]]

In [0]:
pre_transactions

In [0]:
new_transactions = []
new_transactions = pre_transactions.loc[pre_transactions['target'] == 1]


In [0]:
new_transactions

In [0]:
no_transactions = []
no_transactions = pre_transactions.loc[pre_transactions['target'] == 0]



In [0]:
no_transactions

In [0]:
no_transactions.shape

In [0]:
# Shuffle data

from sklearn.utils import shuffle
no_transactions = shuffle(no_transactions)
new_transactions = shuffle(new_transactions)

In [0]:
no_transactions

In [0]:

new_transactions



In [0]:
# for i, j in enumerate(new_transactions):
#     if j == 0:
#         print(new_transactions[0:])

In [0]:
# new_transactions.to_csv('new_transactions.csv', index=False)

In [0]:
# Need to create new model file?
# !touch "/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/model.h5"

## Using k-folds

In [0]:
# Set arrays to values only before running fit
train = train.values
targets = targets.values

## Simple Model

In [0]:
new_transactions

In [0]:
transactions = new_transactions[new_transactions.columns[1:]]
no_transactions = no_transactions[no_transactions.columns[1:]]

In [0]:
no_transactions

In [0]:
transactions

In [0]:
def get_targets(bn_no, bn_yes):
  b = []
  z = np.zeros((2000,1))
  b = np.append(bn_no, z, axis=1)

  c = []
  d = np.ones((2000,1))
  c = np.append(bn_yes, d, axis=1)

  cb = np.concatenate((c, b), axis=0)
  full_array = cb[:,:200]
  print(full_array.shape)

  targets_01 = cb[:, 200]

  targets_01 = pd.DataFrame(data=targets_01, columns=["target"])
  
  return targets_01, full_array

In [0]:
# Batch 1
b1_no = no_transactions[0:2000]
b1_yes = transactions[0:2000]
b1_target, X_1 = get_targets(b1_no, b1_yes)

# Batch 2
b2_no = no_transactions[2000:4000]
b2_yes = transactions[2000:4000]
b2_target, X_2 = get_targets(b2_no, b2_yes)

# Batch 3
b3_no = no_transactions[4000:6000]
b3_yes = transactions[4000:6000]
b3_target, X_3 = get_targets(b3_no, b3_yes)

# Batch 4
b4_no = no_transactions[6000:8000]
b4_yes = transactions[6000:8000]
b4_target, X_4 = get_targets(b4_no, b4_yes)

# Batch 5
b5_no = no_transactions[8000:10000]
b5_yes = transactions[8000:10000]
b5_target, X_5 = get_targets(b5_no, b5_yes)

# Batch 6
b6_no = no_transactions[118000:120000]
b6_yes = transactions[10000:12000]
b6_target, X_6 = get_targets(b6_no, b6_yes)

# Batch 7
b7_no = no_transactions[140000:142000]
b7_yes = transactions[12000:14000]
b7_target, X_7 = get_targets(b7_no, b7_yes)

# Batch 8
b8_no = no_transactions[158000:160000]
b8_yes = transactions[14000:16000]
b8_target, X_8 = get_targets(b8_no, b8_yes)

# Batch 9
b9_no = no_transactions[170000:172000]
b9_yes = transactions[16000:18000]
b9_target, X_9 = get_targets(b9_no, b9_yes)

# Batch 10
b10_no = no_transactions[176000:178000]
b10_yes = transactions[18000:20000]
b10_target, X_10 = get_targets(b10_no, b10_yes)

In [0]:
# Create the h5 file in case there is no such file
!touch "/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/model.h5"

In [0]:
X_concat = np.concatenate((X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10), axis=0)
targets_concat = np.concatenate((b1_target,b2_target,b3_target,b4_target,b5_target,b6_target,b7_target,b8_target,b9_target,b10_target), axis=0)

In [0]:
# MLP for Pima Indians Dataset with 10-fold cross validation
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import StratifiedKFold
from keras.metrics import categorical_accuracy
from keras.models import load_model
# For custom metrics
import keras.backend as K
from keras import regularizers

# load pima indians dataset
# dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = X_concat
Y = targets_concat

model = Sequential()
# model.add(Dense(64, input_dim=200, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
model.add(Dense(512, input_dim=200, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu'))
model.add(Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
# model = load_model('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/model.h5')
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [0]:
# Fit the model
original_histo = model.fit(X, Y, epochs=70, batch_size=100, verbose=1)
model.save('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/model.h5')

## Print the graph

In [0]:
epochs = range(0, 10)
original_val_loss = original_histo.history['acc']

import matplotlib.pyplot as plt

# b+ is for "blue cross"
plt.plot(epochs, original_val_loss, 'b', label='Original model')
# "bo" is for "blue dot"
# plt.plot(epochs, smaller_model_val_loss, 'bo', label='Smaller model')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()

plt.show()

## With K-fold Cross Validation

In [0]:
# MLP for Pima Indians Dataset with 10-fold cross validation
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from keras.metrics import categorical_accuracy
from keras.models import load_model
# For custom metrics
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

cvscores = []

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

X = X_concat
Y = targets_concat
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for trn_idx, tst_idx in kfold.split(X, Y):
  # create model
	model = Sequential()
	model.add(Dense(64, input_dim=200, activation='relu'))
	model.add(Dense(32, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', mean_pred])
	# Fit the model
	original_histo = model.fit(X[trn_idx], Y[trn_idx], epochs=70, batch_size=12, verbose=1)
	model.save('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/model.h5')
	# evaluate the model
	scores = model.evaluate(X[tst_idx], Y[tst_idx], verbose=0)
	print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
	cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))






## Set Submission File

In [0]:
test.shape

In [0]:
test = pd.read_csv('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/test.csv')


In [0]:
test

In [0]:
test = test[test.columns[1:]]

In [0]:
test

In [0]:
# load the model we saved
from keras.models import load_model

model = load_model('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/model.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


classes = model.predict(test, batch_size=128, verbose=1)

In [0]:
classes


In [0]:
classes[0]

In [0]:
final_submission = np.asarray(classes)

In [0]:
final_submission = pd.DataFrame(data=final_submission, columns=["p1"])

In [0]:
final_submission.head()

In [0]:
final_submission

In [0]:
sample_submission.head()

In [0]:
sample_submission = pd.read_csv('/content/drive/My Drive/kaggle/santander-customer-transaction-prediction/sample_submission.csv')
sample_submission.shape

In [0]:
sample_submission['target'] = final_submission['p1']
sample_submission.to_csv('submission.csv', index=False)

In [0]:
sample_submission.shape

In [0]:
sample_submission.head()

##Plot what we have

In [0]:
# np.random.seed(12345678)
# x = np.random.random(10)
# y = 1.6*x + np.random.random(10)

slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
print("slope: %f    intercept: %f" % (slope, intercept))

In [0]:
print("r-squared: %f" % r_value**2)

In [0]:
plt.plot(x, y, 'o', label='original data')
plt.plot(x, intercept + slope*x, 'r', label='fitted line')
plt.legend()
plt.show()

#### TODO 0: get all values that have a target of 1 and try to generate a regression for all 200 variables plus passing the other 80% of data set that has ceros.


#### TODO 1: train with 20098 taregets vs another 20098 random 0 targets ?



In [0]:
features

In [0]:
train_test = pd.concat([train, test], axis =0)



target = train_test['target'].values

test_target = train['target']


In [0]:
param = {'num_leaves': 50,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.05,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 17,
         "metric": 'auc',
         "verbosity": -1}

folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train_test))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_test.values, target)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_test.iloc[trn_idx][features], label=target[trn_idx])
    val_data = lgb.Dataset(train_test.iloc[val_idx][features], label=target[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_test.iloc[val_idx][features], num_iteration=clf.best_iteration)

Let's take a look at the overall oof CV AUC:

In [0]:
metrics.roc_auc_score(target, oof)

AUC of 0.52 is overall not a major issue, but it is slightly over 0.50, which can have an impact when you are chasing the 4th decimal point.

## Lasso Implementation 
https://www.kaggle.com/baogorek/lasso-regression-with-keras